In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

C:\Users\anama\AppData\Local\Temp\ipykernel_13056\3777615979.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


# Lab. Natural Lenguage Processing
### SMS: SPAM or HAM

### Let's prepare the environment

In [5]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

- Read Data for the Fraudulent Email Kaggle Challenge
- Reduce the training set to speead up development. 

In [3]:
## Read Data for the Fraudulent Email Kaggle Challenge
data = pd.read_csv("data/kg_train.csv",encoding='latin-1')

# Reduce the training set to speead up development. 
# Modify for final system
data = data.head(1000)
print(data.shape)
data.fillna("",inplace=True)

(1000, 2)


### Let's divide the training and test set into two partitions

In [4]:
data.head()

,text,label
0,"DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL...",1
1,Will do.,0
2,Nora--Cheryl has emailed dozens of memos about...,0
3,Dear Sir=2FMadam=2C I know that this proposal ...,1
4,fyi,0


In [7]:
# Your code
x= data['text']
y=data['label']

In [8]:
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.2, random_state=42)

## Data Preprocessing

In [15]:
import string
import nltk
from nltk.corpus import stopwords
print(string.punctuation)
print(stopwords.words("english")[100:110])
from nltk.stem.snowball import SnowballStemmer
snowball = SnowballStemmer('english')
import re
from bs4 import BeautifulSoup

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
['here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each']


## Now, we have to clean the html code removing words

- First we remove inline JavaScript/CSS
- Then we remove html comments. This has to be done before removing regular tags since comments can contain '>' characters
- Next we can remove the remaining tags

In [16]:
# Initialize the SnowballStemmer and stopwords
nltk.download('stopwords')
snowball = SnowballStemmer('english')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\anama\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [18]:
# Your code
def clean_text(html_content):
    # Step 1: Remove inline JavaScript/CSS
    cleaned_html = re.sub(r'<script.*?>.*?</script>', '', html_content, flags=re.DOTALL)
    cleaned_html = re.sub(r'<style.*?>.*?</style>', '', cleaned_html, flags=re.DOTALL)

    # Step 2: Remove HTML comments
    cleaned_html = re.sub(r'<!--.*?-->', '', cleaned_html, flags=re.DOTALL)

    # Step 3: Remove remaining HTML tags
    soup = BeautifulSoup(cleaned_html, 'html.parser')
    text = soup.get_text()

    return text

# Apply the cleaning function to the training and test data
x_train_cleaned = x_train.apply(clean_text)
x_test_cleaned = x_test.apply(clean_text)

C:\Users\anama\AppData\Local\Temp\ipykernel_13056\2221887919.py:11: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(cleaned_html, 'html.parser')
C:\Users\anama\AppData\Local\Temp\ipykernel_13056\2221887919.py:11: MarkupResemblesLocatorWarning: The input looks more like a URL than markup. You may want to use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  soup = BeautifulSoup(cleaned_html, 'html.parser')


- Remove all the special characters
    
- Remove numbers
    
- Remove all single characters
 
- Remove single characters from the start

- Substitute multiple spaces with single space

- Remove prefixed 'b'

- Convert to Lowercase

In [19]:
# Your code
def data_preprocessing(text):
    # Remove all special characters
    text = re.sub(r'\W', ' ', text)

    # Remove all single characters
    text = re.sub(r'\s+[a-zA-Z]\s+', ' ', text)

    # Remove single characters from the start
    text = re.sub(r'\^[a-zA-Z]\s+', ' ', text)

    # Remove numbers
    text = re.sub(r'\d+', '', text)

    # Substitute multiple spaces with a single space
    text = re.sub(r'\s+', ' ', text, flags=re.I)

    # Remove prefixed 'b'
    text = re.sub(r'^b\s+', '', text)

    # Convert to lowercase
    text = text.lower()

    return text

def further_clean_text(text):
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    
    # Remove stopwords
    words = text.split()
    cleaned_text = ' '.join(word for word in words if word not in stop_words)
    
    # Stemming
    stemmed_text = ' '.join(snowball.stem(word) for word in cleaned_text.split())
    
    return stemmed_text

# Apply the cleaning functions to the training and test data
x_train_cleaned = x_train.apply(clean_text).apply(data_preprocessing).apply(further_clean_text)
x_test_cleaned = x_test.apply(clean_text).apply(data_preprocessing).apply(further_clean_text)

C:\Users\anama\AppData\Local\Temp\ipykernel_13056\2221887919.py:11: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(cleaned_html, 'html.parser')
C:\Users\anama\AppData\Local\Temp\ipykernel_13056\2221887919.py:11: MarkupResemblesLocatorWarning: The input looks more like a URL than markup. You may want to use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  soup = BeautifulSoup(cleaned_html, 'html.parser')


## Now let's work on removing stopwords
Remove the stopwords.

In [20]:
# Your code
def further_clean_text(text):
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    
    # Remove stopwords
    words = text.split()
    cleaned_text = ' '.join(word for word in words if word not in stop_words)
    
    # Stemming
    stemmed_text = ' '.join(snowball.stem(word) for word in cleaned_text.split())
    
    return stemmed_text

x_train_cleaned = x_train.apply(clean_text).apply(further_clean_text)
x_test_cleaned = x_test.apply(clean_text).apply(further_clean_text)

C:\Users\anama\AppData\Local\Temp\ipykernel_13056\2221887919.py:11: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(cleaned_html, 'html.parser')
C:\Users\anama\AppData\Local\Temp\ipykernel_13056\2221887919.py:11: MarkupResemblesLocatorWarning: The input looks more like a URL than markup. You may want to use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  soup = BeautifulSoup(cleaned_html, 'html.parser')


## Tame Your Text with Lemmatization
Break sentences into words, then use lemmatization to reduce them to their base form (e.g., "running" becomes "run"). See how this creates cleaner data for analysis!

In [21]:
from nltk.stem import WordNetLemmatizer

In [22]:
lemmatizer = WordNetLemmatizer()

In [23]:
# Your code

# Function to perform lemmatization
def lemmatize_text(text):
    # Tokenize text into words
    words = text.split()
    
    # Remove stopwords and lemmatize
    cleaned_words = [lemmatizer.lemmatize(word) for word in words if word.lower() not in stop_words]
    
    # Join cleaned words back into a sentence
    cleaned_text = ' '.join(cleaned_words)
    
    return cleaned_text

# Apply lemmatization to the cleaned text
x_train_cleaned = x_train.apply(lemmatize_text)
x_test_cleaned = x_test.apply(lemmatize_text)

## Bag Of Words
Let's get the 10 top words in ham and spam messages (**EXPLORATORY DATA ANALYSIS**)

In [24]:
from sklearn.feature_extraction.text import CountVectorizer


In [25]:
# Initialize CountVectorizer to create bag of words
vectorizer = CountVectorizer()

# Fit the vectorizer on training data and transform training data
x_train_bow = vectorizer.fit_transform(x_train_cleaned)

# Transform test data using the same vectorizer
x_test_bow = vectorizer.transform(x_test_cleaned)

# Get feature names (words)
feature_names = vectorizer.get_feature_names_out()

# Convert bow matrices to pandas DataFrames for visualization
x_train_bow_df = pd.DataFrame(x_train_bow.toarray(), columns=feature_names)
x_test_bow_df = pd.DataFrame(x_test_bow.toarray(), columns=feature_names)

# Display the bag of words DataFrames
print("Bag of Words for Training Data:")
print(x_train_bow_df)
print("\nBag of Words for Test Data:")
print(x_test_bow_df)

Bag of Words for Training Data:
     00  000  000000  00000e25  00000e251  00000eur  000066  0000ff  000m  \
0     0    0       0         0          0         0       0       0     0   
1     0    0       0         0          0         0       0       0     0   
2     0    0       0         0          0         0       0       0     0   
3     0    0       0         0          0         0       0       0     0   
4     0    0       0         0          0         0       0       0     0   
..   ..  ...     ...       ...        ...       ...     ...     ...   ...   
795   0    0       0         0          0         0       0       0     0   
796   0    0       0         0          0         0       0       0     0   
797   0    0       0         0          0         0       0       0     0   
798   0    0       0         0          0         0       0       0     0   
799   0    0       0         0          0         0       0       0     0   

     000million  ...  â½s  â½t  â½ta  â½te 

## Extra features

In [ ]:
# We add to the original dataframe two additional indicators (money symbols and suspicious words).
money_simbol_list = "|".join(["euro","dollar","pound","€","$"])
suspicious_words = "|".join(["free","cheap","sex","money","account","bank","fund","transfer","transaction","win","deposit","password"])

data_train['money_mark'] = data_train['preprocessed_text'].str.contains(money_simbol_list)*1
data_train['suspicious_words'] = data_train['preprocessed_text'].str.contains(suspicious_words)*1
data_train['text_len'] = data_train['preprocessed_text'].apply(lambda x: len(x)) 

data_val['money_mark'] = data_val['preprocessed_text'].str.contains(money_simbol_list)*1
data_val['suspicious_words'] = data_val['preprocessed_text'].str.contains(suspicious_words)*1
data_val['text_len'] = data_val['preprocessed_text'].apply(lambda x: len(x)) 

data_train.head()

## How would work the Bag of Words with Count Vectorizer concept?

In [ ]:
# Your code

## TD-IDF

- Load the vectorizer

- Vectorize all dataset

- print the shape of the vetorized dataset

In [26]:
# Your code
# Initialize TfidfVectorizer to create TF-IDF vectors
vectorizer = TfidfVectorizer()

# Fit the vectorizer on training data and transform training data
x_train_tfidf = vectorizer.fit_transform(x_train_cleaned)

# Transform test data using the fitted vectorizer
x_test_tfidf = vectorizer.transform(x_test_cleaned)

# Print the shape of the vectorized dataset
print("Shape of Vectorized Training Dataset:", x_train_tfidf.shape)
print("Shape of Vectorized Test Dataset:", x_test_tfidf.shape)

Shape of Vectorized Training Dataset: (800, 23057)
Shape of Vectorized Test Dataset: (200, 23057)


## And the Train a Classifier?

In [29]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score


In [30]:
# Your code
# Initialize the classifier (Logistic Regression in this case)
classifier = LogisticRegression()

# Train the classifier on the TF-IDF transformed training data
classifier.fit(x_train_tfidf, y_train)

# Predict labels for the test set
y_pred = classifier.predict(x_test_tfidf)

# Evaluate the classifier performance
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

# Print classification report (optional)
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

Accuracy: 0.98

Classification Report:
              precision    recall  f1-score   support

           0       0.97      1.00      0.98       125
           1       1.00      0.95      0.97        75

    accuracy                           0.98       200
   macro avg       0.98      0.97      0.98       200
weighted avg       0.98      0.98      0.98       200



### Extra Task - Implement a SPAM/HAM classifier

https://www.kaggle.com/t/b384e34013d54d238490103bc3c360ce

The classifier can not be changed!!! It must be the MultinimialNB with default parameters!

Your task is to find the **best feature representation**.

You can work with teams of two persons (recommended).

In [ ]:
# Your code